In [76]:
import operator 
import math

class KNN():
    def __init__(self, K=5):
        self.K = K
    
    def euclideanDistance(self, vector1, vector2):
        distance = 0
        for a,b in zip(vector1, vector2):
            distance += pow((float(a) - float(b)), 2)
        return math.sqrt(distance)

    def getNeighbors(self, trainingSet, testInstance):
        distances = []
        for x in trainingSet:
            dist = self.euclideanDistance(testInstance.EigenValues, x.EigenValues)
            distances.append((x, dist))
        distances.sort(key=operator.itemgetter(1))
        neighbors = []
        for x in range(self.K):
            neighbors.append(distances[x][0])
        return neighbors

    def getResponse(self, neighbors):
        classVotes = {}
        for x in range(len(neighbors)):
            response = neighbors[x].SuperPop
            if response in classVotes:
                classVotes[response] += 1
            else:
                classVotes[response] = 1
        sortedVotes = sorted(classVotes.iteritems(), key=operator.itemgetter(1), reverse=True)
        return sortedVotes[0][0]

In [111]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import random


ControlPanel = "integrated_call_samples_v3.20130502.ALL.panel"
EVEC = "data/RGN.plus.HapMap.pca.evec"

class Control:
    def __init__(self, row):
        self.ID, self.Pop, self.SuperPop, self.Gender = row
        
class Evec:
    def __init__(self, row):
        self.ID = row[0].split(":")[-1]
        self.Partition = row[-1]
        self.EigenValues = row[1:-1]
        
class Dev:
    def __init__(self):
        self.ControlPanelFil = ControlPanel
        self.EvecFil = EVEC
    def loadControlPanel(self):
        # sample  pop     super_pop       gender
        self.ControlPanel = {}
        fin = open(self.ControlPanelFil, 'rb')
        reader = csv.reader(fin, delimiter='\t')
        header = reader.next()
        for row in reader:
            Indv = Control(row)
            self.ControlPanel[Indv.ID] = Indv
    def loadEigenValue(self):
        fin = open(self.EvecFil, 'rb')
        self.Colors = {'SAS':"green" , 'EAS':"blue", 'AMR':"orange", 'AFR':"purple", 'EUR':"red", 'CASE':"grey"}
        self.SuperPops = {'SAS':[] , 'EAS':[], 'AMR':[], 'AFR':[], 'EUR':[]} #'SAS', 'EAS', 'AMR', 'AFR', 'EUR'
        self.Cases = []
        header = fin.next()
        for line in fin:
            row = line.strip().split()
            evec = Evec(row)
            if evec.Partition == "Case":
                self.Cases.append(evec)
            elif evec.Partition == "Control":
                try:
                    superpop = self.ControlPanel[evec.ID].SuperPop
                    self.SuperPops[superpop].append(evec)
                except KeyError:
                    print "Sample %s marked as control but not in Control Panel" % (evec.ID)
                
        
    def ReduceSuperPop(self):
        self.Colors = {'SAS':"green" , 'EAS':"blue", 'AMR':"orange", 'AFR':"purple", 'EUR':"red", 'CASE':"grey"}
        self.SuperPops = {'SAS':[] , 'EAS':[], 'AMR':[], 'AFR':[], 'EUR':[]} #'SAS', 'EAS', 'AMR', 'AFR', 'EUR'
        for k,v in self.ControlPanel.items():
            superpop = v.SuperPop
            if superpop not in self.SuperPops:
                print "{}'s SuperPop {} is not included in default SuperPop: {}".format(k, superpop, "SAS, EAS, AMR, AFR, EUR")
            else:
                self.SuperPops[superpop].append(k)
    
    def ReadEigenvaluesFromList(self, List, idx1, idx2):
        X, Y = [], []
        for evec in List:
            X.append(evec.EigenValues[idx1])
            Y.append(evec.EigenValues[idx2])
        return X, Y
    
    def Plot(self, OutName):
        with PdfPages('%s.pdf'%OutName, 'wb') as pdf:
            for PCx, PCy in [(1,2),(2,3),(3,4),(4,5)]:
                plt.figure(figsize=(15, 15))
                for k,SuperPop in self.SuperPops.items():
                    color = self.Colors[k]
                    X, Y = self.ReadEigenvaluesFromList(SuperPop, PCx-1, PCy-1)
                    plt.scatter(X, Y, color=color, marker='x', label=k)
                X, Y = self.ReadEigenvaluesFromList(self.Cases, PCx-1, PCy-1)
                plt.scatter(X, Y, color="grey", marker='o', label="Cases")
                plt.xlabel('PC%d'%PCx)
                plt.ylabel('PC%d'%PCy)
                plt.title('PC%d - PC%d'%(PCx,PCy))
                plt.legend(loc='upper right')
                plt.grid(True)
                #plt.show()
                pdf.savefig()
                plt.close()

    def MakeTrainingSetKNN(self):
        res = []
        for k,v in self.SuperPops.items():
            #print k, v
            for evec in v:
                evec.SuperPop = k
                res.append(evec)
        return res
                
    def MarkerWithKNN(self):
        with open("CaseAnnotationKNN.txt", 'wb') as fout:
            knn = KNN(5)
            TrainingSet = self.MakeTrainingSetKNN()
            for case in self.Cases:
                neighbors = knn.getNeighbors(TrainingSet, case)
                response = knn.getResponse(neighbors)
                fout.write("%s\t%s\t%s\n"%(case.ID, response,','.join(case.EigenValues)))
        
    def MakeTrainingSetSVM(self):
        res = []
        for k,v in self.SuperPops.items():
            for evec in v:
                evec.SuperPop = k
                res.append(evec)
        random. shuffle(res)
        Data = [x.EigenValues for x in res]
        Label = [x.SuperPop for x in res]
        return Data, Label
    
    def MarkerWithSVM(self):
        with open("CaseAnnotationSVM.txt", 'wb') as fout:
            TrainingSet, TrainingLabel = self.MakeTrainingSetSVM()
            svm = Support_Vector_Machine(TrainingSet, TrainingLabel)
            svm.fit()
            for case in self.Cases:
                response = svm.predict(case.EigenValues)
                fout.write("%s\t%s\t%s\n"%(case.ID, response,','.join(case.EigenValues)))
    

        

In [112]:
from sklearn import svm
import time

class Support_Vector_Machine:
    def __init__(self, TrainingData, TrainingLabels):
        kernel = 'rbf'
        self.svm = svm.SVC(kernel=kernel, gamma=10)
        self.trainingData = TrainingData
        self.trainingLabels = TrainingLabels

    # train
    def fit(self):
        s_time = time.time()
        print "Fitting Model..."
        self.svm.fit(self.trainingData, self.trainingLabels)
        print "Done with Fitting, used %.4f secs." % (time.time() - s_time)

    def predict(self, case):
        dec = self.svm.predict(case)[0]
        return dec


In [113]:
instance = Dev()
instance.loadControlPanel()
instance.loadEigenValue()
#instance.ReduceSuperPop()
#instance.Plot("RGN")
#instance.MarkerWithKNN()
instance.MarkerWithSVM()

Fitting Model...
Done with Fitting, used 0.0650 secs.


c:\Users\Wang Jiayao\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
c:\Users\Wang Jiayao\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
c:\Users\Wang Jiayao\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  

In [114]:
#Validation
class Evec:
    def __init__(self, row):
        self.ID, self.Superpop = row[0], row[1]
        self.EigenValues = row[2].split(",")

class Validation:
    def __init__(self, fname):
        self.fname = fname
        
    def run(self):
        self.loadCase()
        self.Plot("Test.Annotate.SVM")
        
    def loadCase(self):
        fin = open(self.fname, 'rb')
        self.Colors = {'SAS':"green" , 'EAS':"blue", 'AMR':"orange", 'AFR':"purple", 'EUR':"red", 'CASE':"grey"}
        self.SuperPops = {'SAS':[] , 'EAS':[], 'AMR':[], 'AFR':[], 'EUR':[]} #'SAS', 'EAS', 'AMR', 'AFR', 'EUR'
        for line in fin:
            row = line.strip().split()
            evec = Evec(row)
            try:
                self.SuperPops[evec.Superpop].append(evec)
            except KeyError:
                print "Sample %s marked as %s but not in SuperPop Panel" % (evec.ID, case.Superpop)
    
    def ReadEigenvaluesFromList(self, List, idx1, idx2):
        X, Y = [], []
        for evec in List:
            X.append(evec.EigenValues[idx1])
            Y.append(evec.EigenValues[idx2])
        return X, Y
    
    def Plot(self, OutName):
        with PdfPages('%s.pdf'%OutName, 'wb') as pdf:
            for PCx, PCy in [(1,2),(2,3),(3,4),(4,5)]:
                plt.figure(figsize=(15, 15))
                for k,SuperPop in self.SuperPops.items():
                    color = self.Colors[k]
                    X, Y = self.ReadEigenvaluesFromList(SuperPop, PCx-1, PCy-1)
                    plt.scatter(X, Y, color=color, marker='o', label=k)

                plt.xlabel('PC%d'%PCx)
                plt.ylabel('PC%d'%PCy)
                plt.title('PC%d - PC%d'%(PCx,PCy))
                plt.legend(loc='upper right')
                plt.grid(True)
                #plt.show()
                pdf.savefig()
                plt.close()

test = Validation("CaseAnnotation.txt")
test = Validation("CaseAnnotationSVM.txt")
test.run()     